# Revive it
Checking on air quality in Barcelona so we make it a healthier place.

## Dataset
We're using the official data from the city of Barcelona: https://opendata-ajuntament.barcelona.cat/data/en/dataset/qualitat-aire-detall-bcn
As data up to April 2019 is in a different format, we only use data from April 2019 up to the (currently) most recent data: February 2024.

## Check the data and put it in one document

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Importing stuff

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

2024-04-04 17:03:29.288857: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-04 17:03:29.294327: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-04 17:03:29.344867: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-04 17:03:30.336440: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [33]:
# from revive_it_package.data import detectDelimiter

def detectDelimiter(csv_naam):
    csvFile = f"../raw_data/{csv_naam}"

    with open(csvFile, 'r') as myCsvfile:
        header=myCsvfile.readline()
        if header.find(";")!=-1:
            return ";"
        if header.find(",")!=-1:
            return ","
    #default delimiter (MS Office export)
    return ";"

In [34]:
# Putting data in one excel and saving it for futher use
data = pd.DataFrame()

all_csv = [csv for csv in os.listdir("../raw_data") if csv.endswith(".csv")]

for csv in all_csv:
    print(f"starting {csv}")
    new_df = data

    if detectDelimiter(csv) == ",":
        df = pd.read_csv(f"../raw_data/{csv}")
    else:
        df = pd.read_csv(f"../raw_data/{csv}", delimiter=";")
    data = pd.concat([new_df, df])

data.to_csv("../data/data.csv", index=False)


starting 2022_08_Agost_qualitat_aire_BCN.csv
starting 2023_11_Novembre_qualitat_aire_BCN.csv
starting 2023_12_Desembre_qualitat_aire_BCN.csv
starting 2019_10_Octubre_qualitat_aire_BCN.csv
starting 2023_03_Marc_qualitat_aire_BCN.csv
starting 2019_08_Agost_qualitat_aire_BCN.csv
starting 2023_08_Agost_qualitat_aire_BCN.csv
starting 2023_02_Febrer_qualitat_aire_BCN.csv
starting 2019_06_Juny_qualitat_aire_BCN.csv
starting 2023_10_Octubre_qualitat_aire_BCN.csv
starting 2023_05_Maig_qualitat_aire_BCN.csv
starting 2021_07_Juliol_qualitat_aire_BCN.csv
starting 2021_05_Maig_qualitat_aire_BCN.csv
starting 2021_10_Octubre_qualitat_aire_BCN.csv
starting 2023_04_Abril_qualitat_aire_BCN.csv
starting 2022_11_Novembre_qualitat_aire_BCN.csv
starting 2021_02_Febrer_qualitat_aire_BCN.csv
starting 2023_07_Juliol_qualitat_aire_BCN.csv
starting 2020_02_Febrer_qualitat_aire_BCN.csv
starting 2019_12_Desembre_qualitat_aire_BCN.csv
starting 2021_01_Gener_qualitat_aire_BCN.csv
starting 2021_08_Agost_qualitat_aire

In [35]:
# Uploaded saved excel with all data
df = pd.read_csv("../data/data.csv")
df.head()

,CODI_PROVINCIA,PROVINCIA,CODI_MUNICIPI,MUNICIPI,ESTACIO,CODI_CONTAMINANT,ANY,MES,DIA,H01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,8,Barcelona,19,Barcelona,4,7,2022,8,1,1.0,...,2.0,V,2.0,V,1.0,V,2.0,V,1.0,V
1,8,Barcelona,19,Barcelona,4,7,2022,8,2,1.0,...,3.0,V,2.0,V,1.0,V,1.0,V,2.0,V
2,8,Barcelona,19,Barcelona,4,7,2022,8,3,1.0,...,2.0,V,1.0,V,1.0,V,1.0,V,1.0,V
3,8,Barcelona,19,Barcelona,4,7,2022,8,4,1.0,...,1.0,V,NaN,N,1.0,V,1.0,V,NaN,N
4,8,Barcelona,19,Barcelona,4,7,2022,8,5,1.0,...,1.0,V,1.0,V,1.0,V,1.0,V,1.0,V


In [41]:
# Put column names in lower case
df.columns = [col.lower() for col in df.columns]

# Modify data to timeseries
df.rename(columns={'any' : 'year', 'mes': 'month', 'dia': 'day'}, inplace=True)
df['date'] = pd.to_datetime(df[['year','month','day',]])

# Setting dates as the index
df.set_index(['date'], inplace=True)


,codi_provincia,provincia,codi_municipi,municipi,estacio,codi_contaminant,year,month,day,h01,...,h20,v20,h21,v21,h22,v22,h23,v23,h24,v24
date,,,,,,,,,,,,,,,,,,,,,
2022-08-01,8,Barcelona,19,Barcelona,4,7,2022,8,1,1.0,...,2.0,V,2.0,V,1.0,V,2.0,V,1.0,V
2022-08-02,8,Barcelona,19,Barcelona,4,7,2022,8,2,1.0,...,3.0,V,2.0,V,1.0,V,1.0,V,2.0,V
2022-08-03,8,Barcelona,19,Barcelona,4,7,2022,8,3,1.0,...,2.0,V,1.0,V,1.0,V,1.0,V,1.0,V
2022-08-04,8,Barcelona,19,Barcelona,4,7,2022,8,4,1.0,...,1.0,V,NaN,N,1.0,V,1.0,V,NaN,N
2022-08-05,8,Barcelona,19,Barcelona,4,7,2022,8,5,1.0,...,1.0,V,1.0,V,1.0,V,1.0,V,1.0,V


In [42]:
df.head()

,codi_provincia,provincia,codi_municipi,municipi,estacio,codi_contaminant,year,month,day,h01,...,h20,v20,h21,v21,h22,v22,h23,v23,h24,v24
date,,,,,,,,,,,,,,,,,,,,,
2022-08-01,8,Barcelona,19,Barcelona,4,7,2022,8,1,1.0,...,2.0,V,2.0,V,1.0,V,2.0,V,1.0,V
2022-08-02,8,Barcelona,19,Barcelona,4,7,2022,8,2,1.0,...,3.0,V,2.0,V,1.0,V,1.0,V,2.0,V
2022-08-03,8,Barcelona,19,Barcelona,4,7,2022,8,3,1.0,...,2.0,V,1.0,V,1.0,V,1.0,V,1.0,V
2022-08-04,8,Barcelona,19,Barcelona,4,7,2022,8,4,1.0,...,1.0,V,NaN,N,1.0,V,1.0,V,NaN,N
2022-08-05,8,Barcelona,19,Barcelona,4,7,2022,8,5,1.0,...,1.0,V,1.0,V,1.0,V,1.0,V,1.0,V
